In [1]:
SEED = 666

# Install and import packages

In [2]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import MissingIndicator, SimpleImputer, IterativeImputer, KNNImputer
from category_encoders.cat_boost import CatBoostEncoder

In [58]:
from deepctr.inputs import  SparseFeat, DenseFeat, get_feature_names
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam,RMSprop
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import backend as K
from tensorflow.keras import callbacks
from tensorflow.keras import utils
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import Constant
from deepctr.models import DeepFM
import tensorflow.keras as keras
import tensorflow as tf
import pandas as pd
import numpy as np
import re
import warnings
warnings.simplefilter('ignore')

# Load data

In [4]:
train = pd.read_csv('cat-in-the-dat-ii/train.csv')
test = pd.read_csv('cat-in-the-dat-ii/test.csv')

test["target"] = -1

data = pd.concat([train, test]).reset_index(drop=True)

# Label encode and fillna

In [5]:
def convert_data_to_numeric(df):
    
    bin_3_mapping = {'T':1 , 'F':0}
    bin_4_mapping = {'Y':1 , 'N':0}
    nom_0_mapping = {'Red' : 0, 'Blue' : 1, 'Green' : 2}
    nom_1_mapping = {'Trapezoid' : 0, 'Star' : 1, 'Circle': 2, 'Triangle' : 3, 'Polygon' : 4}
    nom_2_mapping = {'Hamster' : 0 , 'Axolotl' : 1, 'Lion' : 2, 'Dog' : 3, 'Cat' : 4, 'Snake' : 5}
    nom_3_mapping = {'Russia' : 0, 'Canada' : 1, 'Finland' : 2, 'Costa Rica' : 3, 'China' : 4, 'India' : 5}
    nom_4_mapping = {'Bassoon' : 0, 'Theremin' : 1, 'Oboe' : 2, 'Piano' : 3}
    nom_5_mapping = dict(zip((df.nom_5.dropna().unique()), range(len((df.nom_5.dropna().unique())))))
    nom_6_mapping = dict(zip((df.nom_6.dropna().unique()), range(len((df.nom_6.dropna().unique())))))
    nom_7_mapping = dict(zip((df.nom_7.dropna().unique()), range(len((df.nom_7.dropna().unique())))))
    nom_8_mapping = dict(zip((df.nom_8.dropna().unique()), range(len((df.nom_8.dropna().unique())))))
    nom_9_mapping = dict(zip((df.nom_9.dropna().unique()), range(len((df.nom_9.dropna().unique())))))
    ord_1_mapping = {'Novice' : 0, 'Contributor' : 1, 'Expert' : 2, 'Master': 3, 'Grandmaster': 4}
    ord_2_mapping = { 'Freezing': 0, 'Cold': 1, 'Warm' : 2, 'Hot': 3, 'Boiling Hot' : 4, 'Lava Hot' : 5}
    ord_3_mapping = {'a':0, 'b':1, 'c':2 ,'d':3 ,'e':4, 'f':5, 'g':6, 'h':7, 'i':8, 'j':9, 'k':10, 'l':11, 'm':12, 'n':13, 'o':14}
    ord_4_mapping = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'H':7, 'I':8, 'J':9, 'K':10,'L':11,'M':12,
                 'N':13,'O':14,'P':15,'Q':16,'R':17,'S':18,'T':19,'U':20,'V':21,'W':22,'X':23,'Y':24,'Z':25}
    sorted_ord_5 = sorted(df.ord_5.dropna().unique())
    ord_5_mapping = dict(zip(sorted_ord_5, range(len(sorted_ord_5))))

    df['bin_3'] = df.loc[df.bin_3.notnull(), 'bin_3'].map(bin_3_mapping)
    df['bin_4'] = df.loc[df.bin_4.notnull(), 'bin_4'].map(bin_4_mapping)
    df['nom_0'] = df.loc[df.nom_0.notnull(), 'nom_0'].map(nom_0_mapping)
    df['nom_1'] = df.loc[df.nom_1.notnull(), 'nom_1'].map(nom_1_mapping)
    df['nom_2'] = df.loc[df.nom_2.notnull(), 'nom_2'].map(nom_2_mapping)
    df['nom_3'] = df.loc[df.nom_3.notnull(), 'nom_3'].map(nom_3_mapping)
    df['nom_4'] = df.loc[df.nom_4.notnull(), 'nom_4'].map(nom_4_mapping)
    df['nom_5'] = df.loc[df.nom_5.notnull(), 'nom_5'].map(nom_5_mapping)
    df['nom_6'] = df.loc[df.nom_6.notnull(), 'nom_6'].map(nom_6_mapping)
    df['nom_7'] = df.loc[df.nom_7.notnull(), 'nom_7'].map(nom_7_mapping)
    df['nom_8'] = df.loc[df.nom_8.notnull(), 'nom_8'].map(nom_8_mapping)
    df['nom_9'] = df.loc[df.nom_9.notnull(), 'nom_9'].map(nom_9_mapping)
    df['ord_1'] = df.loc[df.ord_1.notnull(), 'ord_1'].map(ord_1_mapping)
    df['ord_2'] = df.loc[df.ord_2.notnull(), 'ord_2'].map(ord_2_mapping)
    df['ord_3'] = df.loc[df.ord_3.notnull(), 'ord_3'].map(ord_3_mapping)
    df['ord_4'] = df.loc[df.ord_4.notnull(), 'ord_4'].map(ord_4_mapping)
    df['ord_5'] = df.loc[df.ord_5.notnull(), 'ord_5'].map(ord_5_mapping)
    
    return df

## Define sparse features

In [6]:
sparse_features = [feat for feat in train.columns if feat not in ['id','target']]

## Keep categories present in train AND test

In [7]:
for col in sparse_features:
    train_unique_values = set(train[col].dropna().unique())
    test_unique_values  = set(test[col].dropna().unique())

    symmetric_difference_values = train_unique_values.symmetric_difference(test_unique_values)
    if symmetric_difference_values:
        print(f'{len(symmetric_difference_values)} values in {col}, {symmetric_difference_values} Replaced with nan')
        data.loc[data[col].isin(symmetric_difference_values), col] = np.nan

1 values in nom_5, {'b3ad70fcb'} Replaced with nan
4 values in nom_6, {'3a121fefb', 'a885aacec', 'f0732a795', 'ee6983c6d'} Replaced with nan
2 values in nom_9, {'3d19cd31d', '1065f10dd'} Replaced with nan


## Fillna

In [8]:
train = data[data.target != -1].reset_index(drop=True)
test  = data[data.target == -1].reset_index(drop=True)

In [9]:
train = train.fillna('NaN')
test = test.fillna('NaN')

## Create sequences and tokenize

In [10]:
sequences_train = train[sparse_features].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
sequences_test = test[sparse_features].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

Using TensorFlow backend.


In [24]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(sequences_train))

In [34]:
sequences_train  = tokenizer.texts_to_sequences(list(sequences_train))
sequences_test  = tokenizer.texts_to_sequences(list(sequences_test))

# Define Keras NLP model

In [36]:
def auc(y_true, y_pred):
    def fallback_auc(y_true, y_pred):
        try:
            return roc_auc_score(y_true, y_pred)
        except:
            return 0.5
    return tf.py_function(fallback_auc, (y_true, y_pred), tf.double)

In [37]:
class CyclicLR(keras.callbacks.Callback):

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1 / (2. ** (x - 1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma ** (x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.

    def clr(self):
        cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size))
        x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x)) * self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x)) * self.scale_fn(
                self.clr_iterations)

    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())

    def on_batch_end(self, epoch, logs=None):

        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        K.set_value(self.model.optimizer.lr, self.clr())


# NLP model

In [38]:
target = ['target']
N_Splits = 2
Verbose = 1
Epochs = 50
BATCH_SIZE = 512

DROPOUT = 0.2
NNLAYERS = (256, 256, 256, 256)
PATIENCE = 5

In [40]:
MAX_SEQUENCE_LENGTH = max([len(seq) for seq in sequences_train + sequences_test])
EMBEDDING_DIM = 10
NUM_WORDS = max([max(seq) for seq in sequences_train + sequences_test])

In [53]:
from keras.preprocessing.sequence import pad_sequences
sequences_train = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)
sequences_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)

In [87]:
def create_model():
    """
    """
    embedding_layer = Embedding(
        NUM_WORDS,
        EMBEDDING_DIM,
        input_length=MAX_SEQUENCE_LENGTH
    )
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    
    x = Conv1D(128, 16, activation='relu')(embedded_sequences)
#     x = MaxPooling1D(8)(x)
#     x = Conv1D(128, 8, activation='relu')(x)
#     x = MaxPooling1D(5)(x)
#     x = Conv1D(128, 5, activation='relu')(x)
    x = GlobalMaxPooling1D()(x)
    x = Dense(128, activation='relu')(x)
    preds = Dense(1, activation='softmax')(x)
    model = Model(sequence_input, preds)
    
    return model

In [88]:
oof_pred_deepfm = np.zeros((len(train), ))
y_pred_deepfm = np.zeros((len(test),))

skf = StratifiedKFold(n_splits=N_Splits, shuffle=True, random_state=SEED)
X_test = sequences_test


for fold, (tr_ind, val_ind) in enumerate(skf.split(train, train[target])):
    
    # Split
    X_train, X_val = np.array(sequences_train)[tr_ind], np.array(sequences_train)[val_ind]
    y_train, y_val = train[target].iloc[tr_ind], train[target].iloc[val_ind]
    
    # Define model
    model = create_model()
    
    opt = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(opt, "binary_crossentropy", metrics=[auc])
    
    # Define callbacks
    es = callbacks.EarlyStopping(
        monitor='val_auc', 
        min_delta=0.0, 
        patience=PATIENCE, 
        verbose=Verbose, 
        mode='max', 
        baseline=None, 
        restore_best_weights=True
    )
    #sb = callbacks.ModelCheckpoint(
     #   './nn_model.w8', save_weights_only=True, save_best_only=True, verbose=Verbose
    #)
#     clr = CyclicLR(base_lr=0.00001 / 100, max_lr = 0.0001, 
#                        step_size= int(1.0*(test.shape[0])/1024) , mode='exp_range',
#                        gamma=1., scale_fn=None, scale_mode='cycle')
    reduce_lr = ReduceLROnPlateau(
        monitor='val_auc', 
        mode='max',
        factor=0.5,
        patience=3, 
        min_lr=1e-7,
        verbose=True,
    )
    
    # Train model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=BATCH_SIZE, 
        epochs=Epochs, 
        verbose=Verbose,
        callbacks=[reduce_lr, es]
    )
    
    # Predict
    val_pred = model.predict(X_val, batch_size=512)
    print(f"validation AUC fold {fold+1} : {round(roc_auc_score(y_val, val_pred), 5)}")
    oof_pred_deepfm[val_ind] = val_pred.ravel()
    y_pred_deepfm += model.predict(X_test, batch_size=512).ravel() / (N_Splits)
    K.clear_session()

Train on 480000 samples, validate on 120000 samples
Epoch 1/50
404480/480000 [========================>.....] - ETA: 3s - loss: 12.4666 - auc: 0.5000

InvalidArgumentError:  indices[490,19] = 5645 is not in [0, 5645)
	 [[node embedding_14/embedding_lookup (defined at <ipython-input-88-8bf17abfcc34>:52) ]] [Op:__inference_keras_scratch_graph_21011]

Function call stack:
keras_scratch_graph


In [ ]:
print(f"OOF AUC : {round(roc_auc_score(train.target.values, oof_pred_deepfm), 5)}")

In [ ]:
target = ['target']
N_Splits = 5
Verbose = 1
Epochs = 50
BATCH_SIZE = 512

DROPOUT = 0.2
NNLAYERS = (256, 256, 256)
PATIENCE = 5

In [ ]:
from tqdm import tqdm_notebook as tqdm

In [ ]:
oof_pred_deepfm = np.zeros((len(train), ))
y_pred_deepfm = np.zeros((len(test),))

skf = StratifiedKFold(n_splits=N_Splits, shuffle=True, random_state=SEED)


for fold, (tr_ind, val_ind) in enumerate(skf.split(train, train[target])):
    
    # Split
    X_train, X_val = train[all_features].iloc[tr_ind], train[all_features].iloc[val_ind]
    y_train, y_val = train[target].iloc[tr_ind], train[target].iloc[val_ind]
    train_model_input = {name:X_train[name] for name in feature_names}
    val_model_input = {name:X_val[name] for name in feature_names}
    test_model_input = {name:test[name] for name in feature_names}
    
    # Define model
    model = DeepFM(sparse_feature_columns, sparse_feature_columns + dense_feature_columns,
                   dnn_hidden_units=NNLAYERS, dnn_dropout=DROPOUT, dnn_use_bn=False, task='binary')
    opt = keras.optimizers.Adam(learning_rate=1e-3)
    model.compile(opt, "binary_crossentropy", metrics=[auc])
    
    # Define callbacks
    es = callbacks.EarlyStopping(
        monitor='val_auc', 
        min_delta=0.0, 
        patience=PATIENCE, 
        verbose=Verbose, 
        mode='max', 
        baseline=None, 
        restore_best_weights=True
    )
    #sb = callbacks.ModelCheckpoint(
     #   './nn_model.w8', save_weights_only=True, save_best_only=True, verbose=Verbose
    #)
#     clr = CyclicLR(base_lr=0.00001 / 100, max_lr = 0.0001, 
#                        step_size= int(1.0*(test.shape[0])/1024) , mode='exp_range',
#                        gamma=1., scale_fn=None, scale_mode='cycle')
    reduce_lr = ReduceLROnPlateau(
        monitor='val_auc', 
        mode='max',
        factor=0.5,
        patience=3, 
        min_lr=1e-7,
        verbose=True,
    )
    
    # Train model
    history = model.fit(
        train_model_input, y_train,
        validation_data=(val_model_input, y_val),
        batch_size=BATCH_SIZE, 
        epochs=Epochs, 
        verbose=Verbose,
        callbacks=[reduce_lr, es]
    )
    
    # Predict
    val_pred = model.predict(val_model_input, batch_size=512)
    print(f"validation AUC fold {fold+1} : {round(roc_auc_score(y_val, val_pred), 5)}")
    oof_pred_deepfm[val_ind] = val_pred.ravel()
    y_pred_deepfm += model.predict(test_model_input, batch_size=512).ravel() / (N_Splits)
    K.clear_session()

In [ ]:
print(f"OOF AUC : {round(roc_auc_score(train.target.values, oof_pred_deepfm), 5)}")

* Deeper
* decrease LR
* back to cyclic lr
* decrease patience reduceLRplateau
* add reg for DNN
* dropout levels
* treat emb dim
* nunique + 1 ?
* pseudo label

# Grid

In [ ]:
from sklearn.model_selection import ParameterSampler
grid_params = {
    'dnn_dropout': [0.0, 0.1, 0.2, 0.3, 0.5],
    'dnn_hidden_units': [(256,256, 256), (256, 256), (128, 128, 128), (128, 128), (512, 512)],
    'emb_dim': [2, 3, 4, 8],
    'lr': [1e-3, 1e-4],
    'cyclic': [True, False],
    'bn': [True, False],
    'l2_reg_linear': [1e-05, 1e-04],
    'l2_reg_embedding': [1e-05, 1e-04], 
    'l2_reg_dnn': [0.0, 1e-05, 1e-04]    
}
list_params = list(ParameterSampler(grid_params,
                                    n_iter=30,
                                    random_state=42))

## 1

model = DeepFM(sparse_feature_columns, sparse_feature_columns + dense_feature_columns,
               dnn_hidden_units=param['dnn_hidden_units'], dnn_dropout=param['dnn_dropout'], 
               dnn_use_bn=param['bn'], task='binary',
               l2_reg_linear=param['l2_reg_linear'], l2_reg_embedding=param['l2_reg_embedding'], 
               l2_reg_dnn=param['l2_reg_dnn'], init_std=0.0001,
               seed=SEED)

In [ ]:
from tqdm import tqdm_notebook as tqdm

In [ ]:
list_params

In [ ]:
par = ['lr', 'l2_reg_linear', 'l2_reg_embedding', 'l2_reg_dnn', 'emb_dim', 'dnn_hidden_units', 'dnn_dropout', 'cyclic', 'bn']
val = [0.0001, 1e-5, 0.0001, 0.0001, 8, (128, 128, 128), 0.5, False, False]


In [ ]:
list_params = [{x: y for x, y in zip(par, val)}]

In [ ]:
target = ['target']
N_Splits = 60
Verbose = 1
Epochs = 50
SEED = 0

In [ ]:
cv_perf = []

for param in tqdm(list_params):
    
    oof_pred_deepfm = np.zeros((len(train), ))
    y_pred_deepfm = np.zeros((len(test),))

    skf = StratifiedKFold(n_splits=N_Splits, shuffle=True, random_state=SEED)
    
    sparse_feature_columns = [SparseFeat(feat, train[feat].nunique() + 1, embedding_dim=param['emb_dim']) 
                          for feat in features]
    dense_feature_columns = [DenseFeat(feat, 1) for feat in features_enc + indicator_cols]

    feature_names = get_feature_names(sparse_feature_columns + dense_feature_columns)

    all_features = features + features_enc + indicator_cols

    for fold, (tr_ind, val_ind) in tqdm(enumerate(skf.split(train, train[target]))):

        # Split
        X_train, X_val = train[all_features].iloc[tr_ind], train[all_features].iloc[val_ind]
        y_train, y_val = train[target].iloc[tr_ind], train[target].iloc[val_ind]
        train_model_input = {name:X_train[name] for name in feature_names}
        val_model_input = {name:X_val[name] for name in feature_names}
        test_model_input = {name:test[name] for name in feature_names}

        # Define model
        model = DeepFM(sparse_feature_columns, sparse_feature_columns + dense_feature_columns,
                       dnn_hidden_units=param['dnn_hidden_units'], dnn_dropout=param['dnn_dropout'], 
                       dnn_use_bn=param['bn'], task='binary',
                       l2_reg_linear=param['l2_reg_linear'], l2_reg_embedding=param['l2_reg_embedding'], 
                       l2_reg_dnn=param['l2_reg_dnn'], init_std=0.0001,
                       seed=SEED)
        opt = keras.optimizers.Adam(learning_rate=param['lr'])
        model.compile(opt, "binary_crossentropy", metrics=[auc])

        # Define callbacks
        es = callbacks.EarlyStopping(
            monitor='val_auc', 
            min_delta=0.0, 
            patience=PATIENCE, 
            verbose=Verbose, 
            mode='max', 
            baseline=None, 
            restore_best_weights=True
        )
        #sb = callbacks.ModelCheckpoint(
         #   './nn_model.w8', save_weights_only=True, save_best_only=True, verbose=Verbose
        #)
        if param['cyclic']:
            reduce_lr = CyclicLR(base_lr=0.00001 / 100, max_lr = 0.0001, 
                           step_size= int(1.0*(test.shape[0])/1024) , mode='exp_range',
                           gamma=1., scale_fn=None, scale_mode='cycle')
        else:
            reduce_lr = ReduceLROnPlateau(
                monitor='val_auc', 
                mode='max',
                factor=0.5,
                patience=3, 
                min_lr=1e-7,
                verbose=True,
            )

        # Train model
        history = model.fit(
            train_model_input, y_train,
            validation_data=(val_model_input, y_val),
            batch_size=BATCH_SIZE, 
            epochs=Epochs, 
            verbose=Verbose,
            callbacks=[reduce_lr, es]
        )

        # Predict
        val_pred = model.predict(val_model_input, batch_size=512)
        print(f"validation AUC fold {fold+1} : {round(roc_auc_score(y_val, val_pred), 5)}")
        oof_pred_deepfm[val_ind] = val_pred.ravel()
        y_pred_deepfm += model.predict(test_model_input, batch_size=512).ravel() / (N_Splits)
        K.clear_session()
    cv_perf.append(round(roc_auc_score(train.target.values, oof_pred_deepfm), 5))        

In [ ]:
print(f"OOF AUC : {round(roc_auc_score(train.target.values, oof_pred_deepfm), 5)}")

In [ ]:
test_idx = test.id.values
submission = pd.DataFrame.from_dict({
    'id': test_idx,
    'target': y_pred_deepfm
})
submission.to_csv("submission_deepfm2_cv_60.csv", index=False)
print("Submission file saved!")

In [ ]:
submission.head()

In [ ]:
sub1 = pd.read_csv('./submission_deepfm_cv_50.csv')

In [ ]:
submission.to_csv("submission_deepfm_stacked.csv", index=False)

In [ ]:
stacked

In [ ]:
cv_perf

In [ ]:
len(cv_perf)

In [ ]:
max(cv_perf)

IDEAS:
* feature hashing for nom 5 - 9
* OHE + RegLog
* OHE + PCA
* Embedding noms ?

## 2



In [ ]:
features_ord = [feat for feat in features if 'ord' in feat]

In [ ]:
sparse_feature_columns = [SparseFeat(feat, train[feat].nunique() + 1, embedding_dim=4) 
                          for feat in features if 'ord' not in feat]
dense_feature_columns = [DenseFeat(feat, 1) for feat in features_enc + indicator_cols + features_ord]

feature_names = get_feature_names(sparse_feature_columns + dense_feature_columns)

all_features = features + features_enc + indicator_cols

In [ ]:
list_params = list(ParameterSampler(grid_params,
                                    n_iter=20,
                                    random_state=667))

In [ ]:
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='once')

In [ ]:
target = ['target']
N_Splits = 5
Verbose = 1
Epochs = 50
BATCH_SIZE = 512

DROPOUT = 0.2
NNLAYERS = (256, 256, 256)
PATIENCE = 5

cv2_perf = []

for param in tqdm(list_params):
    
    
    oof_pred_deepfm = np.zeros((len(train), ))
    y_pred_deepfm = np.zeros((len(test),))

    skf = StratifiedKFold(n_splits=N_Splits, shuffle=True, random_state=SEED)
    
    sparse_feature_columns = [SparseFeat(feat, train[feat].nunique() + 1, embedding_dim=param['emb_dim']) 
                          for feat in features]
    dense_feature_columns = [DenseFeat(feat, 1) for feat in features_enc + indicator_cols]

    feature_names = get_feature_names(sparse_feature_columns + dense_feature_columns)

    all_features = features + features_enc + indicator_cols

    for fold, (tr_ind, val_ind) in enumerate(skf.split(train, train[target])):

        # Split
        X_train, X_val = train[all_features].iloc[tr_ind], train[all_features].iloc[val_ind]
        y_train, y_val = train[target].iloc[tr_ind], train[target].iloc[val_ind]
        train_model_input = {name:X_train[name] for name in feature_names}
        val_model_input = {name:X_val[name] for name in feature_names}
        test_model_input = {name:test[name] for name in feature_names}

        # Define model
        model = DeepFM(sparse_feature_columns, sparse_feature_columns + dense_feature_columns,
                       dnn_hidden_units=param['dnn_hidden_units'], dnn_dropout=param['dnn_dropout'], 
                       dnn_use_bn=param['bn'], task='binary',
                       l2_reg_linear=param['l2_reg_linear'], l2_reg_embedding=param['l2_reg_embedding'], 
                       l2_reg_dnn=param['l2_reg_dnn'], init_std=0.0001,
                       seed=SEED)
        opt = keras.optimizers.Adam(learning_rate=param['lr'])
        model.compile(opt, "binary_crossentropy", metrics=[auc])

        # Define callbacks
        es = callbacks.EarlyStopping(
            monitor='val_auc', 
            min_delta=0.0, 
            patience=PATIENCE, 
            verbose=Verbose, 
            mode='max', 
            baseline=None, 
            restore_best_weights=True
        )
        #sb = callbacks.ModelCheckpoint(
         #   './nn_model.w8', save_weights_only=True, save_best_only=True, verbose=Verbose
        #)
        if param['cyclic']:
            reduce_lr = CyclicLR(base_lr=0.00001 / 100, max_lr = 0.0001, 
                           step_size= int(1.0*(test.shape[0])/1024) , mode='exp_range',
                           gamma=1., scale_fn=None, scale_mode='cycle')
        else:
            reduce_lr = ReduceLROnPlateau(
                monitor='val_auc', 
                mode='max',
                factor=0.5,
                patience=3, 
                min_lr=1e-7,
                verbose=True,
            )

        # Train model
        history = model.fit(
            train_model_input, y_train,
            validation_data=(val_model_input, y_val),
            batch_size=BATCH_SIZE, 
            epochs=Epochs, 
            verbose=Verbose,
            callbacks=[reduce_lr, es]
        )

        # Predict
        val_pred = model.predict(val_model_input, batch_size=512)
        print(f"validation AUC fold {fold+1} : {round(roc_auc_score(y_val, val_pred), 5)}")
        oof_pred_deepfm[val_ind] = val_pred.ravel()
        y_pred_deepfm += model.predict(test_model_input, batch_size=512).ravel() / (N_Splits)
        K.clear_session()
    print(cv2_perf)
    cv2_perf.append(round(roc_auc_score(train.target.values, oof_pred_deepfm), 5))     

In [ ]:
#pd.DataFrame(dict(list_params[np.argmax(cv_perf)])).to_csv('./cv1_perf.csv')
pd.DataFrame(dict(list_params[np.argmax(cv2_perf)])).to_csv('./cv2_perf.csv')

In [ ]:
cv2_perf

In [ ]:
dict(list_params[np.argmax(cv2_perf)])

In [ ]:
max(cv2_perf)

## Grid 20

In [ ]:
target = ['target']
N_Splits = 10
Verbose = 1
Epochs = 50
SEED = 0

In [ ]:
from sklearn.model_selection import ParameterSampler
grid_params = {
    'dnn_dropout': [0.0, 0.1, 0.2],
    'dnn_hidden_units': [(256,), (512,), (256, 256), (512, 512)],
    'linear': [linear_feature_columns, linear_feature_columns + dnn_feature_columns],
    'sparse': [dnn_feature_columns, linear_feature_columns + dnn_feature_columns],
    'batch_size': [128, 256, 512],
}
list_params = list(ParameterSampler(grid_params,
                                    n_iter=20,
                                    random_state=0))

In [ ]:
cv_perf = []

for param in list_params:

    oof_pred_deepfm = np.zeros((len(train), ))
    y_pred_deepfm = np.zeros((len(test),))

    skf = StratifiedKFold(n_splits=N_Splits, shuffle=True, random_state=SEED)
    for fold, (tr_ind, val_ind) in enumerate(skf.split(train, train[target])):
        X_train, X_val = train[features+features_enc+indicator_cols].iloc[tr_ind], train[features+features_enc+indicator_cols].iloc[val_ind]
        y_train, y_val = train[target].iloc[tr_ind], train[target].iloc[val_ind]
        train_model_input = {name:X_train[name] for name in feature_names}
        val_model_input = {name:X_val[name] for name in feature_names}
        test_model_input = {name:test[name] for name in feature_names}
        model = DeepFM(param['linear'], param['sparse'],
                       dnn_hidden_units=param['dnn_hidden_units'], dnn_dropout=param['dnn_dropout'], dnn_use_bn=False, task='binary')
        model.compile("adam", "binary_crossentropy", metrics=[auc], )
        es = callbacks.EarlyStopping(monitor='val_auc', min_delta=0.001, patience=4, verbose=Verbose, mode='max', baseline=None, restore_best_weights=True)
        sb = callbacks.ModelCheckpoint('./nn_model.w8', save_weights_only=True, save_best_only=True, verbose=Verbose)
        clr = CyclicLR(base_lr=0.00001 / 100, max_lr = 0.0001, 
                           step_size= int(1.0*(test.shape[0])/1024) , mode='exp_range',
                           gamma=1., scale_fn=None, scale_mode='cycle')
        history = model.fit(train_model_input, y_train,
                            validation_data=(val_model_input, y_val),
                            batch_size=param['batch_size'], epochs=Epochs, verbose=Verbose,
                            callbacks=[es, sb, clr],)
        model.load_weights('./nn_model.w8')
        val_pred = model.predict(val_model_input, batch_size=param['batch_size'])
        print(f"validation AUC fold {fold+1} : {round(roc_auc_score(y_val, val_pred), 5)}")
        oof_pred_deepfm[val_ind] = val_pred.ravel()
        y_pred_deepfm += model.predict(test_model_input, batch_size=param['batch_size']).ravel() / (N_Splits)
        K.clear_session()
    cv_perf.append(round(roc_auc_score(train.target.values, oof_pred_deepfm), 5))

In [ ]:
cv_perf

In [ ]:
list_params[2]

## CV 50 best model

In [ ]:
target = ['target']
N_Splits = 50
Verbose = 1
Epochs = 50
BATCH_SIZE = 32

In [ ]:
oof_pred_deepfm = np.zeros((len(train), ))
y_pred_deepfm = np.zeros((len(test),))

skf = StratifiedKFold(n_splits=N_Splits, shuffle=True, random_state=SEED)


for fold, (tr_ind, val_ind) in enumerate(skf.split(train, train[target])):
    
    # Split
    X_train, X_val = train[features+features_enc+indicator_cols].iloc[tr_ind], train[features+features_enc+indicator_cols].iloc[val_ind]
    y_train, y_val = train[target].iloc[tr_ind], train[target].iloc[val_ind]
    train_model_input = {name:X_train[name] for name in feature_names}
    val_model_input = {name:X_val[name] for name in feature_names}
    test_model_input = {name:test[name] for name in feature_names}
    
    # Define model
    model = DeepFM(linear_feature_columns, linear_feature_columns + dnn_feature_columns,
                   dnn_hidden_units=(512, 512), dnn_dropout=0.0, dnn_use_bn=False, task='binary')
    model.compile("adam", "binary_crossentropy", metrics=[auc])
    
    # Define callbacks
    es = callbacks.EarlyStopping(monitor='val_auc', min_delta=0.001, patience=4, verbose=Verbose, mode='max', baseline=None, restore_best_weights=True)
    sb = callbacks.ModelCheckpoint('./nn_model.w8', save_weights_only=True, save_best_only=True, verbose=Verbose)
#     clr = CyclicLR(base_lr=0.00001 / 100, max_lr = 0.0001, 
#                        step_size= int(1.0*(test.shape[0])/1024) , mode='exp_range',
#                        gamma=1., scale_fn=None, scale_mode='cycle')
    reduce_lr = ReduceLROnPlateau(monitor='val_auc', factor=0.2,
                                  patience=5, min_lr=0.00001)
    
    # Train model
    history = model.fit(train_model_input, y_train,
                        validation_data=(val_model_input, y_val),
                        batch_size=BATCH_SIZE, epochs=Epochs, verbose=Verbose,
                        callbacks=[es, sb, reduce_lr],)
    model.load_weights('./nn_model.w8')
    
    # Predict
    val_pred = model.predict(val_model_input, batch_size=512)
    print(f"validation AUC fold {fold+1} : {round(roc_auc_score(y_val, val_pred), 5)}")
    oof_pred_deepfm[val_ind] = val_pred.ravel()
    y_pred_deepfm += model.predict(test_model_input, batch_size=512).ravel() / (N_Splits)
    K.clear_session()

In [ ]:
print(f"OOF AUC : {round(roc_auc_score(train.target.values, oof_pred_deepfm), 5)}")

In [ ]:
test_idx = test.id.values
submission = pd.DataFrame.from_dict({
    'id': test_idx,
    'target': y_pred_deepfm
})
submission.to_csv("submission_deepfm_cv_50.csv", index=False)
print("Submission file saved!")

In [ ]:
np.save('oof_pred_deepfm_cv_50.npy',oof_pred_deepfm)
np.save('y_pred_deepfm_cv_50.npy',    y_pred_deepfm)